In [1]:
import sys
import os
import pandas as pd
sys.path.append('/afs/cern.ch/cms/PPD/PdmV/tools/McM/')
from rest import McM

In [16]:
mcm = McM(dev=False)
#mcm.get('requests', 'PPD-RunIIWinter19PFCalib17pLHE-00001', method='get')
#mcm.get('requests', query='prepid=*-RunIIWinter19PFCalib17pLHE-*&status=new')
results = mcm.get('requests', query='tags=20221207_vasarkis_WprimeToENu')
len(results)

[2022-12-18 00:53:43,333][INFO] Using SSO cookie file /afs/cern.ch/user/d/dfu/private/mcm-prod-cookie.txt
[2022-12-18 00:53:43,588][WARNING] Most likely SSO cookie is expired, will remake it after 1 seconds
[2022-12-18 00:53:45,481][INFO] Using SSO cookie file /afs/cern.ch/user/d/dfu/private/mcm-prod-cookie.txt


168

In [14]:
CLEAR=False
if CLEAR:
    for r in results:
        mcm.delete(object_type='requests', object_id=r['_id'])

In [18]:
results[0]

{'total_events': 37500,
 'config_id': [],
 'events_per_lumi': 0,
 'cmssw_release': 'CMSSW_12_4_11_patch3',
 'mcdb_id': 0,
 'transient_output_modules': [[]],
 'sequences': [{'eventcontent': ['RAWSIM', 'LHE'],
   'runsScenarioForMC': '',
   'donotDropOnInput': '',
   'extra': '',
   'slhc': '',
   'runsAndWeightsForMC': '',
   'outputCommand': '',
   'dropDescendant': False,
   'pileup': '',
   'hltProcess': '',
   'index': -1,
   'customise': '',
   'custom_conditions': '',
   'datatier': ['GEN-SIM', 'LHE'],
   'processName': '',
   'inputCommands': '',
   'particle_table': '',
   'conditions': '124X_mcRun3_2022_realistic_postEE_v1',
   'harvesting': '',
   'triggerResultsProcess': '',
   'inputEventContent': '',
   'beamspot': 'Run3RoundOptics25ns13TeVLowSigmaZ',
   'restoreRNDSeeds': '',
   'filtername': '',
   'repacked': '',
   'customise_commands': '',
   'step': ['LHE', 'GEN', 'SIM'],
   'gflash': '',
   'procModifiers': '',
   'magField': '',
   'himix': False,
   'nThreads': 1,


In [19]:
for r in results:
    r['time_event'] = [20]
    r['size_event'] = [600]
    r['fragment'] = r['fragment'].replace('from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *', 'from  Configuration.Generator.MCTunesRun3ECM13p6TeV.PythiaCP5Settings_cfi import *')

In [20]:
results[0]

{'total_events': 37500,
 'config_id': [],
 'events_per_lumi': 0,
 'cmssw_release': 'CMSSW_12_4_11_patch3',
 'mcdb_id': 0,
 'transient_output_modules': [[]],
 'sequences': [{'eventcontent': ['RAWSIM', 'LHE'],
   'runsScenarioForMC': '',
   'donotDropOnInput': '',
   'extra': '',
   'slhc': '',
   'runsAndWeightsForMC': '',
   'outputCommand': '',
   'dropDescendant': False,
   'pileup': '',
   'hltProcess': '',
   'index': -1,
   'customise': '',
   'custom_conditions': '',
   'datatier': ['GEN-SIM', 'LHE'],
   'processName': '',
   'inputCommands': '',
   'particle_table': '',
   'conditions': '124X_mcRun3_2022_realistic_postEE_v1',
   'harvesting': '',
   'triggerResultsProcess': '',
   'inputEventContent': '',
   'beamspot': 'Run3RoundOptics25ns13TeVLowSigmaZ',
   'restoreRNDSeeds': '',
   'filtername': '',
   'repacked': '',
   'customise_commands': '',
   'step': ['LHE', 'GEN', 'SIM'],
   'gflash': '',
   'procModifiers': '',
   'magField': '',
   'himix': False,
   'nThreads': 1,


In [21]:
for r in results:
    output = mcm.update('requests', r)
    if output['results'] == False:
        print(r['_id'],output['message'])